In [2]:
import tensorflow

In [3]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.optimizers import Adam
from keras import backend as K
import matplotlib.pyplot as plt

EPISODES = 400

Using TensorFlow backend.


In [4]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95 #discount rate
        self.epsilon = 1.0 #for exploration
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.model = self._build_model()
    
    def _build_model(self):
        #building the neural network
        model = Sequential()
        model.add(Dense(32, input_dim = 2, activation = 'relu'))
        model.add(Dense(32, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        model.compile(loss = "mse", optimizer = Adam(lr = self.learning_rate))
        return model
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state,done))
        
    def act(self, state):
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_val = self.model.predict(state) #reward value based on current state
        return np.argmax(act_val[0]) #gives left or right
    
    def train(self, batch_size = 32):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            
            if not done:
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f, epochs = 1, verbose = 0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
    def load(self, name):
        self.model.load_weights(name)
    def save(self, name):
        self.model.save_weights(name)

In [5]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print(state_size, action_size)

agent = Agent(state_size, action_size)
agent.model.load_weights('final')
done = False
batch_size = 64
output_dir = 'mountain_car'
scores = 0

2 3


In [6]:


for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1,state_size])
    for time in range(1000):
        env.render()
        action = agent.act(state)
        next_state, reward, done , other_info = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,EPISODES,time,agent.epsilon))
            break
    if(len(agent.memory) > batch_size):
        agent.train(batch_size)
        
env.close()


Game Episode :0/400, High Score:199,Exploration Rate:1.0
Game Episode :1/400, High Score:199,Exploration Rate:0.99
Game Episode :2/400, High Score:199,Exploration Rate:0.99
Game Episode :3/400, High Score:199,Exploration Rate:0.99
Game Episode :4/400, High Score:199,Exploration Rate:0.98
Game Episode :5/400, High Score:199,Exploration Rate:0.98
Game Episode :6/400, High Score:199,Exploration Rate:0.97
Game Episode :7/400, High Score:199,Exploration Rate:0.97
Game Episode :8/400, High Score:199,Exploration Rate:0.96
Game Episode :9/400, High Score:199,Exploration Rate:0.96
Game Episode :10/400, High Score:199,Exploration Rate:0.95
Game Episode :11/400, High Score:199,Exploration Rate:0.95
Game Episode :12/400, High Score:199,Exploration Rate:0.94
Game Episode :13/400, High Score:199,Exploration Rate:0.94
Game Episode :14/400, High Score:199,Exploration Rate:0.93
Game Episode :15/400, High Score:199,Exploration Rate:0.93
Game Episode :16/400, High Score:199,Exploration Rate:0.92
Game Epi

Game Episode :139/400, High Score:199,Exploration Rate:0.5
Game Episode :140/400, High Score:199,Exploration Rate:0.5
Game Episode :141/400, High Score:199,Exploration Rate:0.49
Game Episode :142/400, High Score:199,Exploration Rate:0.49
Game Episode :143/400, High Score:199,Exploration Rate:0.49
Game Episode :144/400, High Score:199,Exploration Rate:0.49
Game Episode :145/400, High Score:199,Exploration Rate:0.48
Game Episode :146/400, High Score:199,Exploration Rate:0.48
Game Episode :147/400, High Score:199,Exploration Rate:0.48
Game Episode :148/400, High Score:199,Exploration Rate:0.48
Game Episode :149/400, High Score:199,Exploration Rate:0.47
Game Episode :150/400, High Score:199,Exploration Rate:0.47
Game Episode :151/400, High Score:199,Exploration Rate:0.47
Game Episode :152/400, High Score:199,Exploration Rate:0.47
Game Episode :153/400, High Score:199,Exploration Rate:0.46
Game Episode :154/400, High Score:199,Exploration Rate:0.46
Game Episode :155/400, High Score:199,Expl

Game Episode :276/400, High Score:199,Exploration Rate:0.25
Game Episode :277/400, High Score:199,Exploration Rate:0.25
Game Episode :278/400, High Score:199,Exploration Rate:0.25
Game Episode :279/400, High Score:199,Exploration Rate:0.25
Game Episode :280/400, High Score:199,Exploration Rate:0.25
Game Episode :281/400, High Score:199,Exploration Rate:0.24
Game Episode :282/400, High Score:199,Exploration Rate:0.24
Game Episode :283/400, High Score:199,Exploration Rate:0.24
Game Episode :284/400, High Score:199,Exploration Rate:0.24
Game Episode :285/400, High Score:199,Exploration Rate:0.24
Game Episode :286/400, High Score:199,Exploration Rate:0.24
Game Episode :287/400, High Score:199,Exploration Rate:0.24
Game Episode :288/400, High Score:199,Exploration Rate:0.24
Game Episode :289/400, High Score:199,Exploration Rate:0.23
Game Episode :290/400, High Score:199,Exploration Rate:0.23
Game Episode :291/400, High Score:199,Exploration Rate:0.23
Game Episode :292/400, High Score:199,Ex

In [7]:
agent.save('final')